<a href="https://colab.research.google.com/github/wa-le/autoscout24-webscrape/blob/master/autoscout24_scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""!pip install bs4
!pip install requests
!pip install pandas
!pip install numpy"""

In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [ ]:
HEADERS = ({'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'})


In [ ]:
# list to store url of all cars on autoscout24.be
all_url = []

In [ ]:
# send request to the 20 accessible pages on the website and scrape the url of each car
# store in list -> "all_url"

for each_page in range(1, 21):
  webpage = requests.get(f"https://www.autoscout24.be/nl/lst?sort=standard&desc=0&ustate=N%2CU&atype=C&cy=B&search_id=10hyyuturim&page={each_page}", headers=HEADERS)
  # webpage.content
  soup = BeautifulSoup(webpage.content, "html.parser")
  links = soup.find_all("a", attrs={'class': 'ListItem_title__znV2I Link_link__pjU1l'})
  for i in range(0, 20):
    all_url.append(links[i].get("href"))

In [ ]:
# all_url now contains links to all the cars on the website
len(all_url)

400

In [ ]:
# Functions to scrape each property

# scrape name
def get_name(soup):
  try:
    name = soup.find('span', attrs={'class':'StageTitle_boldClassifiedInfo__L7JmO'}).text
  except AttributeError:
    name = ""

  return name

# scrape model
def get_model(soup):
  try:
    model = soup.find('span', attrs={'class':'StageTitle_model__pG_6i StageTitle_boldClassifiedInfo__L7JmO'}).text
  except AttributeError:
    model = ""

  return model

# scrape price
def get_price(soup):
  try:
    price = soup.find('span', attrs={'class':'StandardPrice_price__X_zzU'}).text.split(",")[0]
  except AttributeError:
    price = ""

  return price

# scrape kilometers
def get_kilometers(soup):
  try:
    kilometers = soup.find_all('div', attrs={'class':'VehicleOverview_itemContainer__Ol37r'})[0].find('div', attrs={'class':'VehicleOverview_itemText__V1yKT'}).text
  except AttributeError:
    kilometers = ""

  return kilometers

# scrape transmission
def get_transmission(soup):
  try:
    transmission = soup.find_all('div', attrs={'class':'VehicleOverview_itemContainer__Ol37r'})[1].find('div', attrs={'class':'VehicleOverview_itemText__V1yKT'}).text
  except AttributeError:
    transmission = ""

  return transmission

# scrape manufactured_date
def get_manufactured_date(soup):
  try:
    manufactured_date = soup.find_all('div', attrs={'class':'VehicleOverview_itemContainer__Ol37r'})[2].find('div', attrs={'class':'VehicleOverview_itemText__V1yKT'}).text
  except AttributeError:
    manufactured_date = ""

  return manufactured_date

# scrape fuel_type
def get_fuel_type(soup):
  try:
    fuel_type = soup.find_all('div', attrs={'class':'VehicleOverview_itemContainer__Ol37r'})[3].find('div', attrs={'class':'VehicleOverview_itemText__V1yKT'}).text
  except AttributeError:
    fuel_type = ""

  return fuel_type

# scrape power
def get_power(soup):
  try:
    power = soup.find_all('div', attrs={'class':'VehicleOverview_itemContainer__Ol37r'})[4].find('div', attrs={'class':'VehicleOverview_itemText__V1yKT'}).text
  except AttributeError:
    power = ""

  return power

# scrape model
def get_seller(soup):
  try:
    seller = soup.find_all('div', attrs={'class':'VehicleOverview_itemContainer__Ol37r'})[5].find('div', attrs={'class':'VehicleOverview_itemText__V1yKT'}).text
  except AttributeError:
    seller = ""
    
  return seller


In [ ]:
# dict to values in each col
first_data = {
    "name": [],
    "model": [],
    "price": [],
    "kilometers": [],
    "transmission": [],
    "manufactured_date": [],
    "fuel_type": [],
    "power": [],
    "seller": []
}

In [ ]:
# scrape data from each url
for url in all_url:
  webpage = requests.get(f"https://www.autoscout24.be{url}", headers=HEADERS)
  soup = BeautifulSoup(webpage.content, "html.parser")

  first_data['name'].append(get_name(soup))

  first_data['model'].append(get_model(soup))

  first_data['price'].append(get_price(soup))

  first_data['kilometers'].append(get_kilometers(soup))

  first_data['transmission'].append(get_transmission(soup))

  first_data['manufactured_date'].append(get_manufactured_date(soup))

  first_data['fuel_type'].append(get_fuel_type(soup))

  first_data['power'].append(get_power(soup))

  first_data['seller'].append(get_seller(soup))

In [ ]:
len(first_data['fuel_type'])

400

In [ ]:
# create a dataframe using the 'first_data' dict
autoscout24_df = pd.DataFrame.from_dict(first_data)

In [ ]:
autoscout24_df['kilometers'].replace('-', '', inplace=True)
autoscout24_df['manufactured_date'].replace('-', '', inplace=True)
autoscout24_df['fuel_type'].replace('-', '', inplace=True)

In [83]:
autoscout24_df.head(15)

,name,model,price,kilometers,transmission,manufactured_date,fuel_type,power,seller
0,Abarth,695C,€ 30.950,0 km,Manueel,09/2022,Benzine,121 kW (165 PK),Autobedrijf
1,Mercedes-Benz,S 580,€ 158.000,10 km,Automatisch,,Elektrisch/Benzine,380 kW (517 PK),Autobedrijf
2,Porsche,911,€ 249.950,6.000 km,Automatisch,01/2022,Benzine,353 kW (480 PK),Autobedrijf
3,Porsche,Cayman,€ 102.990,9.155 km,Manueel,03/2020,Benzine,309 kW (420 PK),Autobedrijf
4,Ford,Mustang,€ 33.990,49.000 km,Manueel,05/2019,Benzine,233 kW (317 PK),Autobedrijf
5,BMW,640,€ 24.000,116.753 km,Automatisch,10/2013,Benzine,235 kW (320 PK),Autobedrijf
6,Mercedes-Benz,AMG GT,€ 94.750,71.189 km,Automatisch,01/2019,Benzine,270 kW (367 PK),Autobedrijf
7,BMW,M5,€ 59.990,121.730 km,Automatisch,08/2018,Benzine,441 kW (600 PK),Autobedrijf
8,Hyundai,TUCSON,€ 32.700,7.627 km,Automatisch,11/2021,Elektrisch/Benzine,,Autobedrijf
9,Porsche,Boxster,€ 30.900,135.900 km,Automatisch,08/2006,Benzine,217 kW (295 PK),Autobedrijf


In [ ]:
"""
Data Explanation

name - name of car
model - car model
price - car price in Euros
kilometers - amount of kilometers already on the car in KM
transmission - manual or automatic
manufactured_date - year car was manufactured
fuel_type - type of fuel the car uses
power - car power in KW and horsepower(in bracket)
seller - car seller

"""

In [84]:
# save df to csv file
autoscout24_df.to_csv('autoscout24-cars.csv')

In [ ]:
# trying to scrape more data ...
grab_basic_data_title = soup.find_all('div', attrs={'class':'DetailsSection_childrenSection__NQLD7'})[2].find_all('dt', attrs={'class':'DataGrid_defaultDtStyle__yzRR_'})
basic_data_title = [j.text for j in grab_basic_data_title]
print(basic_data_title)

grab_basic_data_value = soup.find_all('div', attrs={'class':'DetailsSection_childrenSection__NQLD7'})[2].find_all('dd', attrs={'class':'DataGrid_defaultDdStyle__29SKf DataGrid_fontBold__r__dO'})
basic_data_value = [j.text for j in grab_basic_data_value]
print(basic_data_value)

In [ ]:
# take length of each list, know how I can iterate through both
# have a dictionary containing all possible properties and then assign values to the ones that have

# iterate through dict and see if title/key in list, get the index and use the index to select its value in the value list and add the value in the value list in dict